# HiTS: Setup Users, Projects and other Metadata associated with an experiment

In [1]:
import sys
sys.path.append('/home/apaulson/repos/smdc_process_data/')

from database_interactions import mysql_cnx as mc
from database_interactions import metadata as md

%config Completer.use_jedi=False

import pandas as pd
import numpy as np
from sqlalchemy import text


# MetaData stores a collection of tables we want to work with
from sqlalchemy import MetaData
# Table reflection gets an existing table from the db that can be queried against; requires a metadata object for holding info (I think)
from sqlalchemy import Table

In [13]:
# start with nightly!
engine=mc.create_mysql_cnx(db='QB3_HiTS_nightly')
# engine=mc.create_mysql_cnx(db='QB3_HiTS')

# check connection
with engine.connect() as conn:
    result=conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


In [3]:
metadata = MetaData()

# Create screening center
use sqlalchemy, kind of clunky but safer because you have to commit separately before it gets saved to db

In [4]:
# create table object to query against for INV_SCREENING CENTER
sc=Table('INV_SCREENING_CENTER', metadata, autoload_with=engine)
print(sc.primary_key.columns[0])
sc_cols=[x.name for x in sc.columns]
sc_cols

INV_SCREENING_CENTER.ISC_ID


['ISC_ID', 'ISC_NAME', 'ISC_URL', 'ISC_INSTITUTION']

In [5]:
# create dict with new info you want to add to the table
new_sc={
    'ISC_ID':'BUCK',
    'ISC_NAME':'Buck Institute Morphology and Imaging Core',
    'ISC_URL':'https://www.buckinstitute.org/',
    'ISC_INSTITUTION':'Buck Institute for Research on Aging',
}

In [6]:
# check table before inserting - safe to print out all rows bc this is a small table
read_table_stmt=text("select * from INV_SCREENING_CENTER")
print(sc_cols)
with engine.connect() as conn:
    for row in conn.execute(read_table_stmt):
        print(row)

['ISC_ID', 'ISC_NAME', 'ISC_URL', 'ISC_INSTITUTION']
('', 'Public Access - Not Affiliated', None, None)
('BUCK', 'Buck Institute Morphology and Imaging Core', 'https://www.buckinstitute.org/', 'Buck Institute for Research on Aging')
('CSC', 'UCSC Chemical Screening Center', 'https://cscenter.pbsci.ucsc.edu/', 'UC Santa Cruz')
('CTAF-HTSF', 'UCB QB3 Cell and Tissue Analysis Facility / High-Throughput Screening Facility', 'https://qb3.berkeley.edu/ctaf-htsf/', 'UC Berkeley')
('GENE', 'Genentech', None, 'Genentech')
('SMDC', 'UCSF Small Molecule Discovery Center', 'https://pharm.ucsf.edu/smdc', 'UC San Francisco')
('STRATEOS', 'Strateos', 'https://strateos.com/', 'Strateos')


In [7]:
from sqlalchemy import insert
# build insert and print it
stmt=sc.insert().values(new_sc)
print(stmt)

INSERT INTO "INV_SCREENING_CENTER" ("ISC_ID", "ISC_NAME", "ISC_URL", "ISC_INSTITUTION") VALUES (:ISC_ID, :ISC_NAME, :ISC_URL, :ISC_INSTITUTION)


In [289]:
# check insert statement, this might throw errors if you did it wrong 
with engine.connect() as conn:
    result=conn.execute(stmt)
# without commit, you can see what *would* happen
result.last_inserted_params()

{'ISC_ID': 'BUCK',
 'ISC_NAME': 'Buck Institute Morphology and Imaging Core',
 'ISC_URL': 'https://www.buckinstitute.org/',
 'ISC_INSTITUTION': 'Buck Institute for Research on Aging'}

In [290]:
# check table object for what was inserted - nothing because you didn't commit
print(sc_cols)
with engine.connect() as conn:
    for row in conn.execute(read_table_stmt):
        print(row)

['ISC_ID', 'ISC_NAME', 'ISC_URL', 'ISC_INSTITUTION']
('', 'Public Access - Not Affiliated', None, None)
('CSC', 'UCSC Chemical Screening Center', 'https://cscenter.pbsci.ucsc.edu/', 'UC Santa Cruz')
('CTAF-HTSF', 'UCB QB3 Cell and Tissue Analysis Facility / High-Throughput Screening Facility', 'https://qb3.berkeley.edu/ctaf-htsf/', 'UC Berkeley')
('GENE', 'Genentech', None, 'Genentech')
('SMDC', 'UCSF Small Molecule Discovery Center', 'https://pharm.ucsf.edu/smdc', 'UC San Francisco')
('STRATEOS', 'Strateos', 'https://strateos.com/', 'Strateos')


In [291]:
# insert and commit - careful to only run this once! better, insert inside an if statement checking if primary key already exists
with engine.connect() as conn:
    result=conn.execute(stmt)
    conn.commit()
result.last_inserted_params()

{'ISC_ID': 'BUCK',
 'ISC_NAME': 'Buck Institute Morphology and Imaging Core',
 'ISC_URL': 'https://www.buckinstitute.org/',
 'ISC_INSTITUTION': 'Buck Institute for Research on Aging'}

In [8]:
# check table object for what was inserted - this time you see the new entry
print(sc_cols)
with engine.connect() as conn:
    for row in conn.execute(read_table_stmt):
        print(row)

['ISC_ID', 'ISC_NAME', 'ISC_URL', 'ISC_INSTITUTION']
('', 'Public Access - Not Affiliated', None, None)
('BUCK', 'Buck Institute Morphology and Imaging Core', 'https://www.buckinstitute.org/', 'Buck Institute for Research on Aging')
('CSC', 'UCSC Chemical Screening Center', 'https://cscenter.pbsci.ucsc.edu/', 'UC Santa Cruz')
('CTAF-HTSF', 'UCB QB3 Cell and Tissue Analysis Facility / High-Throughput Screening Facility', 'https://qb3.berkeley.edu/ctaf-htsf/', 'UC Berkeley')
('GENE', 'Genentech', None, 'Genentech')
('SMDC', 'UCSF Small Molecule Discovery Center', 'https://pharm.ucsf.edu/smdc', 'UC San Francisco')
('STRATEOS', 'Strateos', 'https://strateos.com/', 'Strateos')


# Create users
use pandas which is easier IMO but:
- you can't really double check before committing
- it's harder to identify the primary key, so make sure you're familiar with the database

pandas uses sqlalchemy under the hood

### Directory affiliate (DM_DA_ID)

In [9]:
dir_aff_cat=pd.read_sql_table('DIRECTORY_AFFILIATE_CATEGORY', con=engine)
dir_aff_cat

,DAC_ID,DAC_NAME
0,1,Academic-UC
1,2,Academic/Government
2,3,Corporate
3,4,Unknown
4,5,Personal


In [54]:
# read whole table into memory - don't do this for large tables!
dir_aff=pd.read_sql_table(table_name='DIRECTORY_AFFILIATE', con=engine)
dir_aff.head(2)

,DA_ID,DA_NAME,DA_DAC_ID
0,1,UC San Francisco,1
1,2,UC Santa Cruz,1


In [11]:
# create dataframe with new rows to add to db
new_aff={
    'DA_NAME': 'Buck Institute',
    'DA_DAC_ID':2 # this is directory_affiliate_category = Academic/Government
}
new_aff=pd.DataFrame.from_records([new_aff])
new_aff

,DA_NAME,DA_DAC_ID
0,Buck Institute,2


#### insert with pd.to_sql()
pd.to_sql() inserts and commits; if there's no unique check (like here) you could insert this 100 times with no error. Best to do a check.

    # NEVER skip if_exists='append', this adds new rows
    # default is if_exists='fail' so nothing will work 
    # if_exists='replace' drops and creates a new table, so probably never use this
    
    # index=False also necessary or it tries to insert the index column

In [16]:
import importlib
importlib.reload(md)
dir_aff_table=Table('DIRECTORY_AFFILIATE', metadata, autoload_with=engine)

if md.check_unique(new_aff, dir_aff_table, 'DA_NAME', engine):
    new_aff.to_sql('DIRECTORY_AFFILIATE', con=engine, if_exists='append', index=False)

['DA_ID', 'DA_NAME', 'DA_DAC_ID']
(25, 'Buck Institute', 2)



In [53]:
# check that it was inserted
dir_aff=pd.read_sql_table(table_name='DIRECTORY_AFFILIATE', con=engine)
dir_aff.tail(2)

,DA_ID,DA_NAME,DA_DAC_ID
20,22,Eindhoven University of Technology,2
21,25,Buck Institute,2


### Directory affiliate department (DM_DAD_ID)

In [55]:
dir_dep=pd.read_sql_table('DIRECTORY_AFFILIATE_DEPARTMENT', con=engine)
dir_dep.head()

,DAD_ID,DAD_NAME
0,1,Pharmacy
1,3,Chemistry
2,4,Microbiology and Environmental Toxicology
3,5,Dentistry
4,6,Benioff Children’s Hospital


In [19]:
new_dep={'DAD_NAME':'Morphology and Imaging Core'}
new_dep=pd.DataFrame.from_records([new_dep])
new_dep

,DAD_NAME
0,Morphology and Imaging Core


In [52]:
dir_dep_table=Table('DIRECTORY_AFFILIATE_DEPARTMENT', metadata, autoload_with=engine)

if md.check_unique(new_dep, dir_dep_table, 'DAD_NAME', engine):
    new_dep.to_sql('DIRECTORY_AFFILIATE_DEPARTMENT', con=engine, if_exists='append', index=False)
    
pd.read_sql_table('DIRECTORY_AFFILIATE_DEPARTMENT', con=engine).tail(2)

['DAD_ID', 'DAD_NAME']
(20, 'Morphology and Imaging Core')



,DAD_ID,DAD_NAME
16,19,Cell and Molecular Pharm.
17,20,Morphology and Imaging Core


### Directory member

In [24]:
dir_mem=pd.read_sql_table(table_name='DIRECTORY_MEMBER', con=engine)
dir_mem.columns

Index(['DM_ID', 'DM_FNAME', 'DM_LNAME', 'DM_TITLE', 'DM_WPHONE', 'DM_EXT',
       'DM_EMAIL', 'DM_PP_USERNAME', 'DM_DA_ID', 'DM_DAD_ID', 'DM_DAC_ID',
       'DM_SPONSOR_ID', 'DM_SCREENING_CENTER', 'DM_NOTES', 'DM_STATUS_ID',
       'DM_INS_BY', 'DM_INS_DATE'],
      dtype='object')

In [30]:
# exclude automatically assigned primary key, DM_ID
# additional info is always better - 
new_member={
 'DM_FNAME':"Sima",
 'DM_LNAME':"T",
 'DM_TITLE':"Data Scientist",
 'DM_WPHONE':np.nan,
 'DM_EXT':np.nan,
 'DM_EMAIL':"email@email.org",
 'DM_PP_USERNAME':np.nan,
 'DM_DA_ID':25, # from directory_affiliate
 'DM_DAD_ID':20, # from directory_affiliate_department
 'DM_DAC_ID':2, # from directory_affiliate_category
 'DM_SPONSOR_ID':np.nan,
 'DM_SCREENING_CENTER':"BUCK", # from inv_screening_center
 'DM_NOTES':np.nan,
 'DM_STATUS_ID':1,
 'DM_INS_BY':1, # You can be circular here, like create yourself as #1 and then say ins_by 1
 'DM_INS_DATE':md.string_now()
}
new_members=pd.DataFrame.from_records([new_member])
new_members

,DM_FNAME,DM_LNAME,DM_TITLE,DM_WPHONE,DM_EXT,DM_EMAIL,DM_PP_USERNAME,DM_DA_ID,DM_DAD_ID,DM_DAC_ID,DM_SPONSOR_ID,DM_SCREENING_CENTER,DM_NOTES,DM_STATUS_ID,DM_INS_BY,DM_INS_DATE
0,Sima,T,Data Scientist,NaN,NaN,email@email.org,NaN,25,20,2,NaN,BUCK,NaN,1,1,2024-03-06 16:02:12


In [27]:
dir_mem_table=Table('DIRECTORY_MEMBER', metadata, autoload_with=engine)

In [28]:
if md.check_unique(new_members, dir_mem_table, 'DM_EMAIL', engine):
    new_members.to_sql(name='DIRECTORY_MEMBER', con=engine, if_exists = 'append', index=False)

OK - these DM_EMAIL values are unique.


In [31]:
pd.read_sql_table('DIRECTORY_MEMBER', con=engine).tail(1)

,DM_ID,DM_FNAME,DM_LNAME,DM_TITLE,DM_WPHONE,DM_EXT,DM_EMAIL,DM_PP_USERNAME,DM_DA_ID,DM_DAD_ID,DM_DAC_ID,DM_SPONSOR_ID,DM_SCREENING_CENTER,DM_NOTES,DM_STATUS_ID,DM_INS_BY,DM_INS_DATE
609,643,Sima,T,Data Scientist,None,NaN,email@email.org,None,25.0,20.0,2.0,NaN,BUCK,None,1,569.0,2024-03-06 16:00:53


### Assign roles to member

In [32]:
dir_roles=pd.read_sql_table('DIRECTORY_ROLE', con=engine)
dir_roles

,DR_ID,DR_DESCRIPTION,DR_IS_STAFF
0,1,Principal Investigator,0
1,2,Staff - Biologist,1
2,3,Staff - Chemist,1
3,4,Staff - Informatics,1
4,5,Staff - IT,1
5,6,Investigator,0
6,7,Graduate Student,0
7,8,Undergraduate Student,0
8,9,Intern,0
9,10,Visiting Scientist,0


In [33]:
dm_roles=pd.read_sql_table('DIRECTORY_MEMBER_ROLES', con=engine)
dm_roles.head()

,DR_ID,DM_ID
0,1,9
1,1,18
2,1,19
3,1,22
4,3,25


In [34]:
# create many roles for same person
new_roles=[
    {'DR_ID':1, 'DM_ID':642}, # PI
    {'DR_ID':4, 'DM_ID':642}, # Informatics staff
    {'DR_ID':6, 'DM_ID':642}, # Investigator, aka person running actual experiments
    {'DR_ID':14,'DM_ID':642}, # regular staff
]
new_roles=pd.DataFrame.from_records(new_roles)
new_roles

,DR_ID,DM_ID
0,1,642
1,4,642
2,6,642
3,14,642


In [35]:
# there's not really a unique check for this table, so skip it and be careful to only insert once.
# we could expand the check_unique to accept > 1 column
new_roles.to_sql(name='DIRECTORY_MEMBER_ROLES', con=engine, if_exists='append', index=False)

4

# Create project

In [41]:
# load project table and count rows - ok to load all into memory?
hts_project_table=Table("HTS_PROJECT", metadata, autoload_with=engine)

md.count_rows(hts_project_table, engine) # OK number of rows

Primary key: HP_ID


[(247,)]

In [42]:
# examine table
hts_project=pd.read_sql_table('HTS_PROJECT', con=engine)
print(hts_project.columns)
hts_project.head(2)

Index(['HP_ID', 'HP_CDD_VAULT_PROJECT_ID', 'HP_SDESC', 'HP_LDESC',
       'HP_FUNDING_SOURCE', 'HP_LABOWNER_UID', 'HP_INVESTIG_UID',
       'HP_PRIMARY_CONTACT', 'HP_SECONDARY_CONTACT', 'HP_THERAPEUTIC_AREA',
       'HP_THERAPEUTIC_AREA_ADD', 'HP_TARGET', 'HP_TARGET_DISEASE',
       'HP_TARGET_ORGANISM', 'HP_TARGET_PROTEIN',
       'HP_TARGET_BIOLOGICAL_PROCESS', 'HP_NOTES', 'HP_SCREENING_CENTER',
       'HP_STATUS_ID', 'HP_INS_BY', 'HP_INS_DATE', 'HP_RECHARGE',
       'HP_IS_PUBLIC_DATA', 'HP_IS_PROPRIETARY_DATA'],
      dtype='object')


,HP_ID,HP_CDD_VAULT_PROJECT_ID,HP_SDESC,HP_LDESC,HP_FUNDING_SOURCE,HP_LABOWNER_UID,HP_INVESTIG_UID,HP_PRIMARY_CONTACT,HP_SECONDARY_CONTACT,HP_THERAPEUTIC_AREA,...,HP_TARGET_PROTEIN,HP_TARGET_BIOLOGICAL_PROCESS,HP_NOTES,HP_SCREENING_CENTER,HP_STATUS_ID,HP_INS_BY,HP_INS_DATE,HP_RECHARGE,HP_IS_PUBLIC_DATA,HP_IS_PROPRIETARY_DATA
0,1,10424.0,Thyroid Receptor,None,,9,8,NaN,NaN,neurology,...,None,None,None,SMDC,0,1,2006-08-15 15:58:18,1,0,0
1,2,10595.0,Caspase-1 (2005),None,,17,17,NaN,NaN,inflammation,...,None,None,None,SMDC,0,1,2006-08-15 15:58:18,1,0,0


In [43]:
# create new data
new_project={
    'HP_CDD_VAULT_PROJECT_ID':np.nan, # can be deleted if you want 
    'HP_SDESC':"Buck test data",
    'HP_LDESC':"A test project to practice inserting data to Hits",
    'HP_FUNDING_SOURCE':'internal', # this cannot be null, if you have a chart string or other method to identify how recharge will be paid use that or just say TBD, internal, etc.
    'HP_LABOWNER_UID':642, # from DIRECTORY_MEMBER - PI
    'HP_INVESTIG_UID':642, # person running experiments
    'HP_PRIMARY_CONTACT':642, # staff member helping this person, can be PI or someone else
    'HP_SECONDARY_CONTACT':np.nan,
    'HP_THERAPEUTIC_AREA':"aging",
    'HP_THERAPEUTIC_AREA_ADD':np.nan,
    'HP_TARGET':"aging protein X", # text
    'HP_TARGET_DISEASE':np.nan, # these target details are determined by BAO. leave blank until form is created.
    'HP_TARGET_ORGANISM':np.nan,
    'HP_TARGET_PROTEIN':np.nan,
    'HP_TARGET_BIOLOGICAL_PROCESS':np.nan,
    'HP_NOTES':np.nan,
    'HP_SCREENING_CENTER':"BUCK", # from inv_screening_center
    'HP_STATUS_ID': 1, # 1 means active
    'HP_INS_BY': 1,
    'HP_INS_DATE': md.string_now(),
    'HP_RECHARGE':0, # 0 means it's in your lab and you don't charge for equipment, 1 means outside users
    'HP_IS_PUBLIC_DATA': 0, # 1 means yes, make public (for web app) put everything at 0 for now
    'HP_IS_PROPRIETARY_DATA': 0, # 1 means exclude from cross-database analysis like frequent hitter compounds, etc.
}
new_project=pd.DataFrame.from_records([new_project])
new_project

,HP_CDD_VAULT_PROJECT_ID,HP_SDESC,HP_LDESC,HP_FUNDING_SOURCE,HP_LABOWNER_UID,HP_INVESTIG_UID,HP_PRIMARY_CONTACT,HP_SECONDARY_CONTACT,HP_THERAPEUTIC_AREA,HP_THERAPEUTIC_AREA_ADD,...,HP_TARGET_PROTEIN,HP_TARGET_BIOLOGICAL_PROCESS,HP_NOTES,HP_SCREENING_CENTER,HP_STATUS_ID,HP_INS_BY,HP_INS_DATE,HP_RECHARGE,HP_IS_PUBLIC_DATA,HP_IS_PROPRIETARY_DATA
0,NaN,Buck test data,A test project to practice inserting data to Hits,internal,642,642,642,NaN,aging,NaN,...,NaN,NaN,NaN,BUCK,1,569,2024-03-06 16:03:51,0,0,0


In [273]:
# check and insert
if md.check_unique(new_project, hts_project_table, "HP_SDESC", engine):
    new_project.to_sql('HTS_PROJECT', con=engine, if_exists='append', index=False)

OK - these HP_SDESC values are unique.


In [51]:
pd.read_sql_table('HTS_PROJECT', con=engine).tail(1)

,HP_ID,HP_CDD_VAULT_PROJECT_ID,HP_SDESC,HP_LDESC,HP_FUNDING_SOURCE,HP_LABOWNER_UID,HP_INVESTIG_UID,HP_PRIMARY_CONTACT,HP_SECONDARY_CONTACT,HP_THERAPEUTIC_AREA,...,HP_TARGET_PROTEIN,HP_TARGET_BIOLOGICAL_PROCESS,HP_NOTES,HP_SCREENING_CENTER,HP_STATUS_ID,HP_INS_BY,HP_INS_DATE,HP_RECHARGE,HP_IS_PUBLIC_DATA,HP_IS_PROPRIETARY_DATA
246,320,NaN,Buck test data,A test project to practice inserting data to Hits,internal,642,642,642.0,NaN,aging,...,None,None,None,SMDC,1,569,2024-03-06 14:57:05,0,0,0


# Create assay inside project
HTS_ASSAY is the container for individual data points (HighThroughputScreening). Every single assay type will have an assay listed in HTS_ASSAY
- a primary screening assay will have one HTS_ASSAY entry.
- Follow up dose response assays will have 2 HTS_ASSAY entries (one for raw data and one for DR curves) and an HTL_ASSAY entry (HitToLead). HTL_ASSAY will store dose-response series and curve fit parameters.
- Imaging assays will have an HTS_ASSAY entry for numerical data like number of nuclei in a well, and an HCS_ASSAY entry (high content screening) for actual image data. Imaging assays can have only HTS/HTL if no image data is desired to be saved with the assay.

In [45]:
hts_assay_table=Table('HTS_ASSAY', metadata, autoload_with=engine)
md.count_rows(hts_assay_table, engine) # still ok- pandas OK up to ~50k in which case you want to use chunks (see Pandas documentation).

Primary key: HA_ID


[(2205,)]

In [50]:
hts_assay=pd.read_sql('HTS_ASSAY', con=engine)
print(hts_assay.shape)
print(hts_assay.columns)
hts_assay.head(1)

(2205, 15)
Index(['HA_ID', 'HA_SDESC', 'HA_HP_ID', 'HA_SCREENING_CENTER', 'HA_ACTIVITY',
       'HA_VARIABLE_DATA_TYPE', 'HA_NOTES', 'HA_DB_NOTES', 'HA_DR_RAW',
       'HA_FILTER', 'HA_STATUS_ID', 'HA_INS_BY', 'HA_INS_DATE', 'class',
       'HA_CDD_VAULT_PROTOCOL_ID'],
      dtype='object')


,HA_ID,HA_SDESC,HA_HP_ID,HA_SCREENING_CENTER,HA_ACTIVITY,HA_VARIABLE_DATA_TYPE,HA_NOTES,HA_DB_NOTES,HA_DR_RAW,HA_FILTER,HA_STATUS_ID,HA_INS_BY,HA_INS_DATE,class,HA_CDD_VAULT_PROTOCOL_ID
0,1,TR-1,1,SMDC,I,None,Fluorescein Probe,201401 Recalculation note: add foldchange,NaN,1,1,1.0,NaT,hits.hts.HighThroughputAssay,44621.0


In [47]:
new_assay={
    'HA_SDESC':'Buck test data', # name of assay
    'HA_HP_ID': 320, # from HTS_PROJECT
    'HA_SCREENING_CENTER':"BUCK",
    'HA_ACTIVITY':np.nan, # deprecated
    'HA_VARIABLE_DATA_TYPE':np.nan, # IF curve data, include "SUBSTCONCUM" (for dose response) or "TIMESEC" (for kinetic)
    'HA_NOTES': np.nan, # these are displayed
    'HA_DB_NOTES':np.nan, # these are for reference inside database, like counterscreen assay ID or reorg notes
    'HA_DR_RAW':np.nan, # if dose response, you need one HTS_ASSAY to hold the raw data points and a second one 
                        # to be the actual DR data. for the second assay, write the assay ID of the raw data 
                        # counterpart. Important for inserting data later.
    'HA_FILTER':0, # if this is a duplicate assay with exact same raw data, for example it's analyzed in two different ways, mark as 1.
    'HA_STATUS_ID':1, # 1=active
    'HA_INS_BY': 1,
    'HA_INS_DATE':md.string_now(),
    'class': 'hits.hts.HighThroughputAssay', # for web app; required. The other option is "hits.htl.HitToLeadAssay". Raw DR data points are 
                                             # 'hits.hts.HighThroughputAssay' while the secondary DR HTS_ASSAY entry is "hits.htl.HitToLeadAssay"
    'HA_CDD_VAULT_PROTOCOL_ID': np.nan # can be deleted if you don't want to cross-upload to a CDD vault
}
new_assay=pd.DataFrame.from_records([new_assay])
new_assay

,HA_SDESC,HA_HP_ID,HA_SCREENING_CENTER,HA_ACTIVITY,HA_VARIABLE_DATA_TYPE,HA_NOTES,HA_DB_NOTES,HA_DR_RAW,HA_FILTER,HA_STATUS_ID,HA_INS_BY,HA_INS_DATE,class,HA_CDD_VAULT_PROTOCOL_ID
0,Buck test data,320,BUCK,NaN,NaN,NaN,NaN,NaN,0,1,569,2024-03-06 16:04:26,hits.hts.HighThroughputAssay,NaN


In [346]:
if md.check_unique(new_assay, hts_assay_table, 'HA_SDESC', engine):
    new_assay.to_sql('HTS_ASSAY', con=engine, if_exists='append', index=False)

OK - these HA_SDESC values are unique.


In [49]:
pd.read_sql_table('HTS_ASSAY', con=engine).tail(1)

,HA_ID,HA_SDESC,HA_HP_ID,HA_SCREENING_CENTER,HA_ACTIVITY,HA_VARIABLE_DATA_TYPE,HA_NOTES,HA_DB_NOTES,HA_DR_RAW,HA_FILTER,HA_STATUS_ID,HA_INS_BY,HA_INS_DATE,class,HA_CDD_VAULT_PROTOCOL_ID
2204,2568,Buck test data,320,BUCK,None,None,None,None,NaN,0,1,569.0,2024-03-06 15:43:26,hits.hts.HighThroughputAssay,NaN


## Other assay examples

### DR assay

In [58]:
# inserting this into nightly is fine because it's going to be overwritten so you can play around. inserting into the real db should only happen if you're creating a real assay.

new_assay_raw={
    'HA_SDESC':'Buck test data DR Raw', # name of assay
    'HA_HP_ID': 320, # from HTS_PROJECT
    'HA_SCREENING_CENTER':"BUCK",
    'HA_ACTIVITY':np.nan, # deprecated
    'HA_VARIABLE_DATA_TYPE':'SUBSTCONCUM', # IF curve data, include "SUBSTCONCUM" (for dose response) or "TIMESEC" (for kinetic)
    'HA_NOTES': np.nan, # these are displayed on web app
    'HA_DB_NOTES':np.nan, # these are for reference inside database, like counterscreen assay ID or reorg notes
    'HA_DR_RAW':np.nan, # if dose response, you need one HTS_ASSAY to hold the raw data points and a second one 
                        # to be the actual DR data. for the second assay, write the assay ID of the raw data 
                        # counterpart. Important for inserting data later.
    'HA_FILTER':0, # if this is a duplicate assay with exact same raw data, for example it's analyzed in two different ways, mark as 1.
    'HA_STATUS_ID':1, # 1=active
    'HA_INS_BY': 1,
    'HA_INS_DATE':md.string_now(),
    'class': 'hits.hts.HighThroughputAssay', # for web app; required. The other option is "hits.htl.HitToLeadAssay". Raw DR data points are 
                                             # 'hits.hts.HighThroughputAssay' while the secondary DR HTS_ASSAY entry is "hits.htl.HitToLeadAssay"
    'HA_CDD_VAULT_PROTOCOL_ID': np.nan # can be deleted if you don't want to cross-upload to a CDD vault
}
new_assay_raw=pd.DataFrame.from_records([new_assay_raw])
if md.check_unique(new_assay_raw, hts_assay_table, 'HA_SDESC',engine):
    new_assay_raw.to_sql('HTS_ASSAY', con=engine, if_exists='append', index=False)

OK - these HA_SDESC values are unique.


In [59]:
# note the assay id here for the second DR entry
pd.read_sql('HTS_ASSAY', con=engine).tail(1)

,HA_ID,HA_SDESC,HA_HP_ID,HA_SCREENING_CENTER,HA_ACTIVITY,HA_VARIABLE_DATA_TYPE,HA_NOTES,HA_DB_NOTES,HA_DR_RAW,HA_FILTER,HA_STATUS_ID,HA_INS_BY,HA_INS_DATE,class,HA_CDD_VAULT_PROTOCOL_ID
2205,2570,Buck test data DR Raw,320,BUCK,None,SUBSTCONCUM,None,None,NaN,0,1,1.0,2024-03-06 16:13:08,hits.hts.HighThroughputAssay,NaN


In [61]:
new_assay_dr={
    'HA_SDESC':'Buck test data DR', # name of assay
    'HA_HP_ID': 320, # from HTS_PROJECT
    'HA_SCREENING_CENTER':"BUCK",
    'HA_ACTIVITY':np.nan, # deprecated
    'HA_VARIABLE_DATA_TYPE':'SUBSTCONCUM', # IF curve data, include "SUBSTCONCUM" (for dose response) or "TIMESEC" (for kinetic)
    'HA_NOTES': np.nan, # these are displayed on web app
    'HA_DB_NOTES':np.nan, # these are for reference inside database, like counterscreen assay ID or reorg notes
    'HA_DR_RAW':2570, # if dose response, you need one HTS_ASSAY to hold the raw data points and a second one 
                        # to be the actual DR data. for the second assay, write the assay ID of the raw data 
                        # counterpart. Important for inserting data later.
    'HA_FILTER':0, # if this is a duplicate assay with exact same raw data, for example it's analyzed in two different ways, mark as 1.
    'HA_STATUS_ID':1, # 1=active
    'HA_INS_BY': 1,
    'HA_INS_DATE':md.string_now(),
    'class': 'hits.htl.HitToLeadAssay', # for web app; required. The other option is "hits.htl.HitToLeadAssay". Raw DR data points are 
                                             # 'hits.hts.HighThroughputAssay' while the secondary DR HTS_ASSAY entry is "hits.htl.HitToLeadAssay"
    'HA_CDD_VAULT_PROTOCOL_ID': np.nan # can be deleted if you don't want to cross-upload to a CDD vault
}
new_assay_dr=pd.DataFrame.from_records([new_assay_dr])
if md.check_unique(new_assay_dr, hts_assay_table, 'HA_SDESC',engine):
    new_assay_dr.to_sql('HTS_ASSAY', con=engine, if_exists='append', index=False)
pd.read_sql('HTS_ASSAY', con=engine).tail(2)

['HA_ID', 'HA_SDESC', 'HA_HP_ID', 'HA_SCREENING_CENTER', 'HA_ACTIVITY', 'HA_VARIABLE_DATA_TYPE', 'HA_NOTES', 'HA_DB_NOTES', 'HA_DR_RAW', 'HA_FILTER', 'HA_STATUS_ID', 'HA_INS_BY', 'HA_INS_DATE', 'class', 'HA_CDD_VAULT_PROTOCOL_ID']
(2571, 'Buck test data DR', 320, 'BUCK', None, 'SUBSTCONCUM', None, None, 2570, 0, 1, 1, datetime.datetime(2024, 3, 6, 16, 14, 39), 'hits.htl.HitToLeadAssay', None)



,HA_ID,HA_SDESC,HA_HP_ID,HA_SCREENING_CENTER,HA_ACTIVITY,HA_VARIABLE_DATA_TYPE,HA_NOTES,HA_DB_NOTES,HA_DR_RAW,HA_FILTER,HA_STATUS_ID,HA_INS_BY,HA_INS_DATE,class,HA_CDD_VAULT_PROTOCOL_ID
2205,2570,Buck test data DR Raw,320,BUCK,None,SUBSTCONCUM,None,None,NaN,0,1,1.0,2024-03-06 16:13:08,hits.hts.HighThroughputAssay,NaN
2206,2571,Buck test data DR,320,BUCK,None,SUBSTCONCUM,None,None,2570.0,0,1,1.0,2024-03-06 16:14:39,hits.htl.HitToLeadAssay,NaN


#### HTL_ASSAY
- you must insert this at the same time as inserting the entry into HTS_ASSAY; you need the HLA_ID primary key available to match with the HA_ID primary key value.
- manually insert the primary key value to skip numbers.

In [64]:
htl_assay_table=Table('HTL_ASSAY', metadata, autoload_with=engine)
md.count_rows(htl_assay_table, engine)

Primary key: HLA_ID


[(576,)]

In [66]:
htl_assay=pd.read_sql('HTL_ASSAY', con=engine)
print(htl_assay.columns)
htl_assay.tail(1)

Index(['HLA_ID', 'HLA_NAME', 'HLA_DESC', 'HLA_NOTES', 'HLA_VARIABLE_DATA_TYPE',
       'HLA_HP_ID', 'HLA_SCREENING_CENTER', 'HLA_HA_ID', 'HLA_STATUS_ID',
       'HLA_INS_BY', 'HLA_INS_DATE', 'class'],
      dtype='object')


,HLA_ID,HLA_NAME,HLA_DESC,HLA_NOTES,HLA_VARIABLE_DATA_TYPE,HLA_HP_ID,HLA_SCREENING_CENTER,HLA_HA_ID,HLA_STATUS_ID,HLA_INS_BY,HLA_INS_DATE,class
575,2567,Rac1 P29S targetC18 MSDR,None,None,SUBSTCONCUM,319,SMDC,2566.0,1,610,2024-02-20 14:32:18,hits.htl.HitToLeadAssay


In [67]:
new_assay_dr

,HA_SDESC,HA_HP_ID,HA_SCREENING_CENTER,HA_ACTIVITY,HA_VARIABLE_DATA_TYPE,HA_NOTES,HA_DB_NOTES,HA_DR_RAW,HA_FILTER,HA_STATUS_ID,HA_INS_BY,HA_INS_DATE,class,HA_CDD_VAULT_PROTOCOL_ID
0,Buck test data DR,320,BUCK,NaN,SUBSTCONCUM,NaN,NaN,2570,0,1,1,2024-03-06 16:15:06,hits.htl.HitToLeadAssay,NaN


In [74]:
# take everything from your previous df
new_htl={
    'HLA_ID':2571, # this is the second entry HA_ID
    'HLA_NAME':new_assay_dr.HA_SDESC.iloc[0],
    'HLA_DESC':np.nan,
    'HLA_NOTES':np.nan,
    'HLA_VARIABLE_DATA_TYPE':new_assay_dr.HA_VARIABLE_DATA_TYPE.iloc[0],
    'HLA_HP_ID':new_assay_dr.HA_HP_ID.iloc[0],
    'HLA_SCREENING_CENTER':new_assay_dr.HA_SCREENING_CENTER.iloc[0],
    'HLA_HA_ID':new_assay_dr.HA_DR_RAW.iloc[0],
    'HLA_STATUS_ID':1,
    'HLA_INS_BY':1,
    'HLA_INS_DATE':md.string_now(),
    'class':new_assay_dr['class'].iloc[0]
}
new_htl=pd.DataFrame([new_htl])
new_htl

,HLA_ID,HLA_NAME,HLA_DESC,HLA_NOTES,HLA_VARIABLE_DATA_TYPE,HLA_HP_ID,HLA_SCREENING_CENTER,HLA_HA_ID,HLA_STATUS_ID,HLA_INS_BY,HLA_INS_DATE,class
0,2571,Buck test data DR,NaN,NaN,SUBSTCONCUM,320,BUCK,2570,1,1,2024-03-06 16:38:03,hits.htl.HitToLeadAssay


In [75]:
if md.check_unique(new_htl, htl_assay_table, 'HLA_ID', engine):
    new_htl.to_sql('HTL_ASSAY', con=engine, if_exists='append', index=False)
pd.read_sql('HTL_ASSAY', con=engine).tail(1)

['HLA_ID', 'HLA_NAME', 'HLA_DESC', 'HLA_NOTES', 'HLA_VARIABLE_DATA_TYPE', 'HLA_HP_ID', 'HLA_SCREENING_CENTER', 'HLA_HA_ID', 'HLA_STATUS_ID', 'HLA_INS_BY', 'HLA_INS_DATE', 'class']
(2571, 'Buck test data DR', None, None, 'SUBSTCONCUM', 320, 'BUCK', 2570, 1, 1, datetime.datetime(2024, 3, 6, 16, 31, 39), 'hits.htl.HitToLeadAssay')



,HLA_ID,HLA_NAME,HLA_DESC,HLA_NOTES,HLA_VARIABLE_DATA_TYPE,HLA_HP_ID,HLA_SCREENING_CENTER,HLA_HA_ID,HLA_STATUS_ID,HLA_INS_BY,HLA_INS_DATE,class
576,2571,Buck test data DR,None,None,SUBSTCONCUM,320,BUCK,2570.0,1,1,2024-03-06 16:31:39,hits.htl.HitToLeadAssay


#### HCS_EXPERIMENT

This table works similarly to `HTL_ASSAY`. A new entry can be created IF there is actual image data to be associated with:
- an HTS_ASSAY primary screening experiment
- an HTL_ASSAY raw data experiment housed in HTS_ASSAY. In this case, the raw data would have `HA_ID` 2570, not 2571.

# Upload data to assay

next notebook...